In [1]:
# pip install bert-embedding

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import tensor,LongTensor
from torch.utils.data import DataLoader
import numpy

### Working with Bert

In [3]:
from bert_embedding import BertEmbedding

/home/phani/anaconda2/envs/kaggle/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
bert_abstract = """We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers.
 Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations by jointly conditioning on both left and right context in all layers.
 As a result, the pre-trained BERT representations can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task-specific architecture modifications. 
BERT is conceptually simple and empirically powerful. 
It obtains new state-of-the-art results on eleven natural language processing tasks, including pushing the GLUE benchmark to 80.4% (7.6% absolute improvement), MultiNLI accuracy to 86.7 (5.6% absolute improvement) and the SQuAD v1.1 question answering Test F1 to 93.2 (1.5% absolute improvement), outperforming human performance by 2.0%."""
sentences = bert_abstract.split('\n')
bert_embedding = BertEmbedding(max_seq_length=200)
result = bert_embedding(sentences)

In [5]:
def bert_to_tensor(result):
    embeds=result[0][1]
    tens=torch.empty(len(embeds),768)
    idx=0
    for k in embeds:
        tens[idx]=tensor(k)
        idx+=1
    return tens

In [6]:
# def bert_to_tensor(result,nonzero=False):
#     embeds=list(zip(*result))[1]
#     x,y =len(np.nonzero(embeds)[0]),768
#     if nonzero:x=len((embeds))
#     tens=torch.empty(x,y)
#     idx=0
#     for k in embeds:
#         if k:
#             tens[idx]=tensor(k[0])
#             idx+=1
#     return tens

### Data Preparation

In [7]:
from pickle import load
from pickle import dump
from collections import Counter

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

def to_vocab(lines):
	vocab = Counter()
	for line in lines:
		tokens = line.split()
		vocab.update(tokens)
	return vocab


def save_clean_sentences(sentences, filename):
    dump(sentences, open(filename, 'wb'))
    print('Saved: %s' % filename)


### Subset Dataset

In [ ]:
en_lines = load_clean_sentences('english_vocab.pkl')
fr_lines = load_clean_sentences('french_vocab.pkl')

In [ ]:
en_subset_sentences=[]
fr_subset_sentences=[]
for idx,sentence in enumerate(fr_lines) :
    if len(sentence)<=150 and len(en_lines[idx])<=150:
        en_subset_sentences.append(en_lines[idx])
        fr_subset_sentences.append(fr_lines[idx])

In [ ]:
len(en_subset_sentences),len(fr_subset_sentences)

In [ ]:
len(to_vocab(en_subset_sentences)),len(to_vocab(fr_subset_sentences))

In [ ]:
max(fr_subset_sentences,key=len)

In [ ]:
fr_vocab_to_idx={}
fr_idx_to_vocab={}
fr_vocab_to_idx['PAD']=0
fr_idx_to_vocab[0]='PAD'
fr_vocab_to_idx['unk']=1
fr_idx_to_vocab[1]='unk'
idx=2
for word in fr_vocab:
    fr_vocab_to_idx[word]=idx
    fr_idx_to_vocab[idx]=word
    idx+=1
    

In [ ]:
en_vocab_to_idx={}
en_idx_to_vocab={}
en_vocab_to_idx['PAD']=0
en_idx_to_vocab[0]='PAD'
en_vocab_to_idx['unk']=1
en_idx_to_vocab[1]='unk'
idx=2
for word in en_vocab:
    en_vocab_to_idx[word]=idx
    en_idx_to_vocab[idx]=word
    idx+=1

In [ ]:
en_sentences_tokens=[]
for sentence in en_subset_sentences:
    new_sent=[en_vocab_to_idx[word] for word in sentence.split()]
    en_sentences_tokens.append(new_sent)

In [ ]:
fr_sentences_tokens=[]
for sentence in fr_subset_sentences:
    new_sent=[fr_vocab_to_idx[word] for word in sentence.split()]
    fr_sentences_tokens.append(new_sent)

In [ ]:
len(fr_sentences_tokens)

In [ ]:
fr_sentences_tokens[1]

In [ ]:
en_sentences_tokens[1]

### Train set and valid set

In [ ]:
np.random.seed(42)
idxs=np.random.permutation(len(en_subset_sentences))
border=int(0.8*len(idxs))
train_idxs=idxs[:border]
valid_idxs=idxs[border:]

In [ ]:
en_subset_sentences_train=[]
en_subset_sentences_valid=[]
fr_sentences_tokens_train=[]
fr_sentences_tokens_valid=[]
for idx in train_idxs:
    en_subset_sentences_train.append(en_subset_sentences[idx])
    fr_sentences_tokens_train.append(fr_sentences_tokens[idx])
for idx in valid_idxs:
    en_subset_sentences_valid.append(en_subset_sentences[idx])
    fr_sentences_tokens_valid.append(fr_sentences_tokens[idx])

In [ ]:
len(en_subset_sentences_train)+len(en_subset_sentences_valid)

In [ ]:
en_subset_sentences_train[0]

In [ ]:
save_clean_sentences(en_subset_sentences_train,"en_train.pkl")
save_clean_sentences(en_subset_sentences_valid,"en_valid.pkl")
save_clean_sentences(fr_sentences_tokens_train,"fr_train.pkl")
save_clean_sentences(fr_sentences_tokens_valid,"fr_valid.pkl")


### Load Train and Valid sets

In [8]:
en_subset_sentences_train=load_clean_sentences("en_train.pkl")
en_subset_sentences_valid=load_clean_sentences("en_valid.pkl")
fr_sentences_tokens_train=load_clean_sentences("fr_train.pkl")
fr_sentences_tokens_valid=load_clean_sentences("fr_valid.pkl")

### Dataset and Data Loader

In [9]:
from torch import LongTensor

In [10]:
class Dataset:
    def __init__(self, x,y):
        self.x = x
        self.y = y
        
    def __getitem__(self, idx):
        en_sequence = self.x[idx]
        fr_sequence = self.y[idx]
        return en_sequence,fr_sequence
    def __len__(self):
        return len(self.x)


def collatebatch(batch):
    batch_size = len(batch)
    seqs, tags, lengths = list(zip(*batch))
    max_length = max(lengths)

    padded_seqs = torch.zeros((batch_size, max_length), dtype=torch.long)
    padded_tags = torch.zeros((batch_size, max_length), dtype=torch.long)
    for idx, length in enumerate(lengths):
        padded_seqs[idx, 0:length] = tensor(seqs[idx][0:length])
        padded_tags[idx, 0:length] = tensor(tags[idx][0:length])
    return padded_seqs, padded_tags

def seq2seq_collate(samples,pad_first=False,backwards=False):
    pad_idx=0
    max_len_x,max_len_y = max([len(s[0].split()) for s in samples]),max([len(s[1]) for s in samples])
    res_x = torch.zeros(len(samples), max_len_x).long() + pad_idx
    res_y = torch.zeros(len(samples), max_len_y).long() + pad_idx
    if backwards: pad_first = not pad_first
    for i,s in enumerate(samples):
        if pad_first: 
            res_x[i,-len(s[0]):],res_y[i,-len(s[1]):] = LongTensor(s[0]),LongTensor(s[1])
        else:         
            res_x[i,:len(s[0]):],res_y[i,:len(s[1]):] = LongTensor(s[0]),LongTensor(s[1])
    if backwards: res_x,res_y = res_x.flip(1),res_y.flip(1)
    return res_x,res_y

def seq2seq_collate_bert(samples,pad_first=False,backwards=False):
    pad_idx=0
    max_len_x,max_len_y = max([len(s[0].split()) for s in samples]),max([len(s[1]) for s in samples])
    res_x = torch.zeros(len(samples), max_len_x,768).float() + pad_idx
    res_y = torch.zeros(len(samples), max_len_y).long() + pad_idx
    for i,s in enumerate(samples):
        vec=bert_to_tensor(bert_embedding([s[0]]))
        res_x[i,:len(s[0].split())] =vec 
        res_y[i,:len(s[1]):] = LongTensor(s[1])
    return res_x,res_y


In [11]:
max(en_subset_sentences_valid,key=lambda x:len(x.split(" ")))

'thar ceann ghrupa uen a uachtarain ta tu i do unk den teach seo le unk anois agus ta tu ar unk de na unk is mo a bhfuil unk unk ar obair an unk seo'

In [12]:
# vec=bert_to_tensor(bert_embedding(en_subset_sentences[0].replace(" ","")),True)
# vec.shape

In [13]:
def get_dls(dataset,collate_fn, bs):
    dataloader = DataLoader(dataset, bs, collate_fn=collate_fn, num_workers=1)
    return dataloader

In [14]:
train_dataset=Dataset(en_subset_sentences_train,fr_sentences_tokens_train)
valid_dataset=Dataset(en_subset_sentences_valid,fr_sentences_tokens_valid)

In [15]:
train_dl=get_dls(train_dataset,seq2seq_collate_bert,2)
valid_dl=get_dls(valid_dataset,seq2seq_collate_bert,8)

In [16]:
# tens1=next(iter(train_dl))


In [17]:
# tens1[0].shape

### Model 

In [18]:
# ##### Max french sentence length is 36 words.
# so the final output layer should be of size(58802,36)

In [19]:
def conv3x3(in_planes, out_planes, stride=1, groups=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, groups=groups, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

In [20]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


In [21]:
basemodel3=nn.Sequential(BasicBlock(1,64),nn.Conv2d(64,512,3,padding=1),nn.AdaptiveMaxPool3d((1,36,58802)))

In [22]:
# vec=basemodel3(tens1[0].unsqueeze(1))
# vec.shape

In [23]:
# vec=vec.reshape(2,36,-1)
# vec.shape

In [24]:
def seq2seq_loss(input, target):
    bs,sl = target.size()
    bs_in,sl_in,nc = input.size()
    if sl>sl_in: input = F.pad(input, (0,0,0,0,0,sl-sl_in))
    input = input[:,:sl]
    return F.cross_entropy(input.contiguous().view(-1,nc), target.view(-1))#, ignore_index=1)

In [25]:
# seq2seq_loss(vec,tens1[1])

### Training_loop

In [29]:
basemodel3=nn.Sequential(BasicBlock(1,64),nn.Conv2d(64,512,3,padding=1),nn.AdaptiveMaxPool3d((1,36,58802)))

In [30]:
opt=torch.optim.SGD(basemodel3.parameters(),lr=0.1,momentum=0.9)

In [31]:
for epoch in range(1):
    for x,y in train_dl:
        input=basemodel3(x.unsqueeze(1))
        input=input.reshape(x.shape[0],36,-1)
        loss=seq2seq_loss(input,y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        print(loss.item())
    

11.297380447387695
11.202016830444336
10.813753128051758
10.423664093017578
8.97403335571289
9.302059173583984
8.15353775024414
10.114811897277832
10.68896770477295
11.102411270141602
10.23232650756836
9.585531234741211
8.245253562927246
10.2781982421875
7.997336387634277


Process Process-2:
Traceback (most recent call last):
  File "/home/phani/anaconda2/envs/kaggle/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/phani/anaconda2/envs/kaggle/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/phani/anaconda2/envs/kaggle/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/home/phani/anaconda2/envs/kaggle/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/home/phani/anaconda2/envs/kaggle/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/home/phani/anaconda2/envs/kaggle/lib/python3.6/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/home/phani/anaconda2/envs/kaggle/lib/python3.6/multiprocessing/conne

KeyboardInterrupt: 